In [ ]:
from google import genai
from google.genai import types
from dotenv import load_dotenv
import os
import time
import random

# Load environment variables once at module level
load_dotenv()

class ResilientClient:
    """
    A wrapper for the Google GenAI client that provides persistent memory 
    and automatic API key rotation upon resource exhaustion.
    """

    def __init__(self, api_keys, model="gemini-2.5-flash-lite", verbose=True):
        self.keys = [k for k in api_keys if k] # Filter invalid keys immediately
        self.model = model
        self.verbose = verbose
        self.current_key_idx = 0
        self.history = [] 
        
        if not self.keys:
            raise ValueError("No valid API keys provided.")
            
        self._init_client()

    @classmethod
    def from_env(cls):
        """Factory method to initialize client using standard environment variables."""
        keys = [
            os.getenv("DEFAULT"),
            os.getenv("HACK1"),
            os.getenv("HACK2"),
            os.getenv("HACK3")
        ]
        return cls(keys)

    def _init_client(self):
        self.client = genai.Client(api_key=self.keys[self.current_key_idx])

    def _rotate_key(self, reason):
        prev_idx = self.current_key_idx
        self.current_key_idx = (self.current_key_idx + 1) % len(self.keys)
        
        if self.verbose:
            print(f"[System] Key index {prev_idx} unavailable ({reason}). Switching to index {self.current_key_idx}.")
        
        self._init_client()

    def chat(self, user_input):
        """
        Sends a message to the model with automatic retry and key rotation logic.
        """
        attempts = 0
        while attempts < len(self.keys):
            try:
                # Re-create session with persistent history
                chat_session = self.client.chats.create(
                    model=self.model,
                    history=self.history
                )
                
                response = chat_session.send_message(user_input)
                
                # Manual history persistence
                self.history.append(types.Content(
                    role="user", parts=[types.Part(text=user_input)]
                ))
                self.history.append(types.Content(
                    role="model", parts=[types.Part(text=response.text)]
                ))
                
                return response.text

            except Exception as e:
                error_msg = str(e).upper()
                
                # 503 Service Unavailable - Backoff and retry same key
                if "503" in error_msg or "UNAVAILABLE" in error_msg:
                    wait_time = (0.5 * (2 ** attempts)) + random.random()
                    if self.verbose:
                        print(f"[System] Service busy. Retrying in {wait_time:.2f}s...")
                    time.sleep(wait_time)
                    attempts += 1
                    continue 
                
                # 429 Quota Exhausted - Rotate key
                elif "429" in error_msg or "RESOURCE_EXHAUSTED" in error_msg:
                    self._rotate_key("Quota Exhausted")
                    attempts += 1 
                    continue
                
                else:
                    return f"System Error: {e}"
        
        return "System Notification: All API keys are currently unavailable."

    def start_interactive_session(self):
        """Starts a blocking terminal session for testing/debugging."""
        print(f"System: Session started with {len(self.keys)} keys. Type 'quit' to exit.")
        
        while True:
            try:
                user_text = input(f"[Key {self.current_key_idx}] User: ")
                
                if user_text.strip().lower() in ["quit", "exit"]:
                    print("System: Terminating session.")
                    break
                
                if not user_text.strip():
                    continue

                response = self.chat(user_text)
                print(f"Model: {response}\n")

            except KeyboardInterrupt:
                print("\nSystem: Terminating session.")
                break


# --- MAIN EXECUTION ---
if __name__ == "__main__":
    try:
        # Initialize and run in one line
        client = ResilientClient.from_env()
        client.start_interactive_session()
    except ValueError as e:
        print(f"Fatal Error: {e}")